In [21]:
import psutil
import os
from collections import Counter
from typing import List, Tuple
import re
import ujson as json  # Usar ujson para un análisis JSON más rápido
from concurrent.futures import ThreadPoolExecutor

# Compilar la expresión regular una sola vez
pattern = re.compile(r'@(\w+)')

# Función para procesar un grupo de líneas y extraer nombres de usuario
def process_lines(lines: List[str]) -> Counter:
    usernames = Counter()
    for line in lines:
        try:
            data = json.loads(line.strip())
            if 'content' in data:
                content = data['content']
                # Extraer los nombres de usuario
                usernames.update(pattern.findall(content))
        except json.JSONDecodeError:
            continue
    return usernames

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    chunk_size = 30  # Tamaño del lote de líneas para procesar
    
    total_counter = Counter()
    
    # Usar ThreadPoolExecutor para paralelizar el procesamiento de líneas
    with ThreadPoolExecutor(max_workers=8) as executor:  # Ajustar basado en el número de núcleos de CPU
        with open(file_path, 'r') as file:
            # Leer y procesar el archivo en bloques
            chunk = []
            for line in file:
                chunk.append(line)
                if len(chunk) >= chunk_size:
                    future = executor.submit(process_lines, chunk)
                    total_counter.update(future.result())
                    chunk = []
            # Procesar el último bloque si hay líneas restantes
            if chunk:
                future = executor.submit(process_lines, chunk)
                total_counter.update(future.result())
    
    # Top 10 nombres de usuario
    top_10_usernames = total_counter.most_common(10)
    
    return top_10_usernames

# Ruta del archivo JSON a procesar
file_path = '/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4-large.json'
top_10_usernames = q3_time(file_path)
print(top_10_usernames)


[('narendramodi', 22610), ('Kisanektamorcha', 18360), ('RakeshTikaitBKU', 16390), ('PMOIndia', 14220), ('RahulGandhi', 11250), ('GretaThunberg', 10460), ('RaviSinghKA', 10150), ('rihanna', 9720), ('UNHumanRights', 9620), ('meenaharris', 9250)]


In [20]:
import psutil
import os
from collections import Counter
from typing import List, Tuple
import re
import ujson as json  # Usar ujson para un análisis JSON más rápido
from concurrent.futures import ThreadPoolExecutor

# Compilar la expresión regular una sola vez
pattern = re.compile(r'@(\w+)')

# Función para obtener el uso actual de memoria en MB
def memory_usage() -> float:
    process = psutil.Process(os.getpid())
    mem_info = process.memory_info()
    return mem_info.rss / 1024 / 1024  # Convertir a MB

# Función para procesar un grupo de líneas y extraer nombres de usuario
def process_lines(lines: List[str]) -> Counter:
    usernames = Counter()
    for line in lines:
        try:
            data = json.loads(line.strip())
            if 'content' in data:
                content = data['content']
                # Extraer los nombres de usuario
                usernames.update(pattern.findall(content))
        except json.JSONDecodeError:
            continue
    return usernames

def q3_time(file_path: str) -> List[Tuple[str, int]]:
    print(f"Uso de memoria antes de cargar el archivo: {memory_usage()} MB")
    
    chunk_size = 30  # Tamaño del lote de líneas para procesar
    
    total_counter = Counter()
    
    # Usar ThreadPoolExecutor para paralelizar el procesamiento de líneas
    with ThreadPoolExecutor(max_workers=8) as executor:  # Ajustar basado en el número de núcleos de CPU
        with open(file_path, 'r') as file:
            # Leer y procesar el archivo en bloques
            chunk = []
            for line in file:
                chunk.append(line)
                if len(chunk) >= chunk_size:
                    future = executor.submit(process_lines, chunk)
                    total_counter.update(future.result())
                    chunk = []
            # Procesar el último bloque si hay líneas restantes
            if chunk:
                future = executor.submit(process_lines, chunk)
                total_counter.update(future.result())
    
    print(f"Uso de memoria después de procesar los datos: {memory_usage()} MB")
    
    # Top 10 nombres de usuario
    top_10_usernames = total_counter.most_common(10)
    
    print(f"Uso de memoria después de contar los nombres de usuario: {memory_usage()} MB")
    
    return top_10_usernames

# Ruta del archivo JSON a procesar
file_path = '/Users/juanignaciomagarinoscastro/Downloads/farmers-protest-tweets-2021-2-4-large.json'
q3_time(file_path)


Uso de memoria antes de cargar el archivo: 325.5625 MB
Uso de memoria después de procesar los datos: 240.78125 MB
Uso de memoria después de contar los nombres de usuario: 240.78125 MB


[('narendramodi', 22610),
 ('Kisanektamorcha', 18360),
 ('RakeshTikaitBKU', 16390),
 ('PMOIndia', 14220),
 ('RahulGandhi', 11250),
 ('GretaThunberg', 10460),
 ('RaviSinghKA', 10150),
 ('rihanna', 9720),
 ('UNHumanRights', 9620),
 ('meenaharris', 9250)]